# Imports

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#defining dates
import datetime as dt
import time

#download stock data
import yfinance as yf

#file handling
import os
from os import listdir
from os.path import isfile, join

#additional plotting tools
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

#make plotly work in Jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter("ignore")


# Constants

In [43]:
PATH = "./stock_data/"

#Start end date constants
S_DATE = "2017-04-16"
E_DATE = "2022-04-16"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

# Get Dataframe from CSV

In [44]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col = 0)
    except FileNotFoundError:
        print("File Doesn't exist")
    else:
        return df


# Plot Bollinger Bands

In [45]:
def plot_bollinger_bands(df,ticker):
    #creating plot figure
    fig = go.Figure()
    #creating candle, arguments are values for the candle
    candle = go.Candlestick(x=df.index, open=df['Open'],
        high = df['High'], low = df['Low'], close = df['Close'], name = 'Candlestick')

    #graphing upper band, 75% opacity
    upper_line = go.Scatter(x=df.index, y=df['upper_band'],
    line=dict(color='rgba(250, 0, 0, 0.75)',
    width = 1), name = 'Upper Band')

    #graphing SMA
    moving_average = go.Scatter(x=df.index, y=df['moving_average'],
    line=dict(color='rgba(0, 0, 250, 0.75)',
    width = 0.7), name = '20 Day SMA')

    #graphing lower band
    lower_line = go.Scatter(x=df.index, y=df['lower_band'],
    line=dict(color='rgba(0, 250, 0, 0.75)',
    width = 1), name = 'Lower Band')

    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(moving_average)
    fig.add_trace(lower_line)

    #range slider allows us to zoom in and out of our plot
    fig.update_xaxes(title = "Date", rangeslider_visible = True)
    fig.update_yaxes(title = "Price") 

    fig.update_layout(title=ticker + " Bollinger Bands",
    height = 1000, width = 1000, showlegend=True)

    fig.show()

# Plot Ichimoku

In [46]:
#change color between red and green depending on the span line crosses for the cloud
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0, 250, 0, 0.4)'
    else:
        return 'rgba(250, 0, 0, 0.4)'

In [47]:
def get_Ichimoku(df):

    #creating candle, arguments are values for the candle
    candle = go.Candlestick(x=df.index, open=df['Open'],
    high = df['High'], low = df['Low'], 
    close = df['Close'], name = 'Candlestick')

    #checks if spanA is greater than spanB to determine color of cloud
    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()
    df = df.groupby('group')

    #putting each dataframe in list
    dfs = []
    for name, data in df:
        dfs.append(data)

    for df in dfs:
        fig.add_traces(go.Scatter(x = df.index, y = df.SpanA,
        line = dict(color = 'rgba(0, 0, 0, 0)')))
    
        fig.add_traces(go.Scatter(x = df.index, y = df.SpanB,
        line = dict(color = 'rgba(0, 0, 0, 0)'),
        fill = 'tonexty',
        fillcolor = get_fill_color(df['label'].iloc[0])))

    baseline = go.Scatter(x = df1.index, y = df1['Baseline'],
    line = dict(color = 'pink', width = 2), name = 'Baseline')

    conversion = go.Scatter(x = df1.index, y = df1['Conversion'],
    line = dict(color = 'black', width = 1), name = 'Conversion')

    lagging = go.Scatter(x = df1.index, y = df1['Lagging'],
    line = dict(color = 'purple', width = 1), name = 'Lagging')

    span_A = go.Scatter(x = df1.index, y = df1['SpanA'],
    line = dict(color = 'green', width = 1, dash = 'dot'), name = 'Span A')

    span_B = go.Scatter(x = df1.index, y = df1['SpanB'],
    line = dict(color = 'red', width = 1, dash = 'dot'), name = 'Span B')

    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_A)
    fig.add_trace(span_B)

    fig.update_layout(height = 1000, width = 1500, showlegend=True)

    fig.show()



# Plots

The Ichimoku (One Look) is considered an all in one indicator that shows support and resistance levels, as well as momentum and trend direction. It computes a "cloud" that attempts to forecast where the price may find support or resistance. The Ichimoku Cloud comprises of 5 lines:
* Conversion Line (Tenkan-sen): Nine-period average - Represents support, resistance, and reversals. Used for short term action
* Baseline (Kijun-sen): 26-period average - Represents support, resistance, and confirms trend changes. Used to evaluate medium term trends. Called the baseline because it lags the price.
* Leading Span A (Senkou A): Average of the Conversion and Base Value over 26 period - Identify future areas of support and resistance
* Leading Span B (Senkou B): Average of Conversion and Base Line over 52 period - Identify future support and resistance
* Lagging Span (Chikou): Lagging closing price - Price shifted back 26 periods. Shows possible support and resistance
* Cloud (Kumo): Space between Span A and B. Represents the divergence in price evolution

When price is above the cloud, the trend is bullish and vice versa. The trend is strengthened if the cloud is moving in the same direction as the price. 

In [49]:
#test bollinger bands
test_df = get_stock_df_from_csv("SPY")
plot_bollinger_bands(test_df, "SPY")

#test ichimoku
#test_df = get_stock_df_from_csv("SPY")
#get_Ichimoku(test_df)

